# Pre-processing and Filtering

This notebook takes as input a folder with AHN point cloud tiles (assumed to be 1x1 km), and performs a number of pre-processing and filtering steps. The output is a reduced point cloud containing probable trees and other "unknown" elements.

In [ ]:
import set_path

import numpy as np
import laspy
import pathlib
import geopandas as gpd
import shapely.geometry as sg
from pathlib import Path

from tqdm.notebook import tqdm
tqdm.pandas()

from upcp.utils import ahn_utils
from upcp.utils.interpolation import FastGridInterpolator
from upcp.region_growing.label_connected_comp import LabelConnectedComp

import gvl.helper_functions as helpers

In [ ]:
import warnings  # temporary, to supress deprecationwarnings from shapely
warnings.filterwarnings('ignore')

## Settings

In [ ]:
BASE_FOLDER = pathlib.Path('../datasets/AHN4')

# Input: AHN subtiles created using notebook "0. LAS Splitter.ipynb"
input_dir = BASE_FOLDER / 'AMS_subtiles_1000'
output_dir = BASE_FOLDER / 'AMS_subtiles_1000_reduced'

# DTM corresponding to AHN subtiles, stored as .npz, created using notebook "0. LAS Splitter.ipynb"
ahn_dtm_folder = BASE_FOLDER / 'npz_subtiles_1000'

# This reference file contains known tree locations, if available. 
# The geopackage contains a 'geometry' column with geopandas POINT coordinates.
# tree_ref_file = '../datasets/validation/tree_locations_amsterdam_xy_rijksdriehoek.gpkg'

MIN_HAG = 2.5  # Minimum height above ground in meters

tree_filter = {'grid_size': 0.5,
               'min_component_size': 50,
               'minmax_hag': 3.5,
               'max_nz_flat': 0.85,
               'min_width': 1.0,
               'max_refl': -5.,
               'max_ampl': 9.,
               'min_nor': 1.5,
               'nor_perc': 80}  # TODO

# AHN classification
AHN_OTHER = 1
AHN_GROUND = 2
AHN_BUILDING = 6
AHN_WATER = 9
AHN_ARTIFACT = 26

# Our classification
UNKNOWN = 0
TREE = 1
NOISE = 2
OTHER = 3

In [ ]:
# Create output folder
pathlib.Path(output_dir).mkdir(parents=True, exist_ok=True)

# Create DTM reader
ahn_reader = ahn_utils.NPZReader(ahn_dtm_folder, caching=False)

# # Load tree locations
# if Path(tree_ref_file).is_file():
#     # We use XY Rijksdriehoek coordinates (epsg:28992), You can use epsg:4326 for wgs84
#     tree_gdf = gpd.read_file(tree_ref_file, crs='epsg:28992')
# else:
#     print('Reference file with known tree locations is not available.')

## Main loop

In [ ]:
input_files = list(pathlib.Path(input_dir).glob('ahn4*.laz'))

In [ ]:
# Check existing output files (ignore this cell to re-run for all tiles)
existing_files = list(pathlib.Path(output_dir).glob('trees*.laz'))
existing_codes = {helpers.get_tilecode_from_filename(file.name)
                  for file in existing_files}

input_files = [file for file in input_files
               if helpers.get_tilecode_from_filename(file.name) not in existing_codes]

In [ ]:
pbar = tqdm(input_files, unit='file', smoothing=0)

for file in pbar:
    tilecode = helpers.get_tilecode_from_filename(file.name)
    pbar.set_postfix_str(tilecode)

    # Load LAS data
    las = laspy.read(file)
    points_xyz = np.vstack((las.x, las.y, las.z)).T

    # # Get trees inside LAS bounding box
    # if Path(tree_ref_file).is_file():
    #     bbox = sg.box(las.header.min[0], las.header.min[1], las.header.max[0], las.header.max[1], ccw=True)
    #     trees_in_bbox = tree_gdf[tree_gdf.within(bbox)]
    #     tree_points = list(trees_in_bbox['geometry'].values)


    ### Reduce the point cloud

    # Use only AHN_OTHER class
    mask = (las.classification == AHN_OTHER)

    # Remove points close to ground
    ground_z = ahn_reader.interpolate(tilecode,
                                      points_xyz[mask])
    height_mask = (points_xyz[mask, 2] - ground_z >= MIN_HAG) | np.isnan(ground_z)

    # Get scalar fields of masked cloud
    orig_idx = np.where(mask)[0][height_mask]
    xyz = points_xyz[orig_idx]
    refl = las.Reflectance[orig_idx]
    ampl = las.Amplitude[orig_idx]
    nor = las.number_of_returns[orig_idx]

    # New scalar fields
    hag = np.around(xyz[:,2] - ground_z[height_mask], decimals=2)
    normals = np.around(helpers.calculate_normals(xyz), decimals=2)

    ### Label the point cloud

    # Init masks
    tree_mask = np.ones(len(xyz), dtype=bool) 
    new_hag = np.copy(hag) # Because we index and manipulate this array in a for loop

    ## Label "tree" clusters based on ground truth tree points
    lcc = LabelConnectedComp(grid_size=tree_filter['grid_size'],
                             min_component_size=tree_filter['min_component_size'])
    point_components = lcc.get_components(xyz)

    # Noise filter
    tree_mask[point_components == -1] = False

    cc_labels = np.unique(point_components)
    cc_labels = set(cc_labels).difference((-1,))

    # Iterate over the clusters
    for cc in tqdm(cc_labels, smoothing=0, leave=False):
        # select points that belong to the cluster
        cc_mask = (point_components == cc)

        # The height above ground values are unknown when a tree is partially above water. 
        # To "standardize" the HAG values per tree, we correct them based on the average HAG for each tree cluster.
        # If a cluster has no valid HAG values, the NAP (z-coordinate) value is used instead.
        cc_z = xyz[cc_mask][:, 2]
        cc_hag = hag[cc_mask]
        if np.isnan(cc_hag).all():
            cc_offset = 0.
        else:
            cc_offset = np.nanmean(cc_hag) - np.mean(cc_z)
        cc_hag = cc_z + cc_offset
        hag[cc_mask] = cc_hag


        ## Filters

        # If most of the points point up, it's not a tree.
        if np.abs(normals[:,2][cc_mask]).mean() > tree_filter['max_nz_flat']:
            tree_mask[cc_mask] = False

        # TODO come up with something clever for x/y flatness
        # Do a similar thing with the x normals
        # elif normals[:,0][cc_mask].mean() < 0.03 and normals[:,0][cc_mask].mean() > -0.03:
        #     tree_mask[cc_mask] = False

        # If reflectance is too high, it's not a tree.
        elif np.mean(refl[cc_mask]) > tree_filter['max_refl']:
            tree_mask[cc_mask] = False

        # If amplitude is too high, it's not a tree.
        elif np.mean(ampl[cc_mask]) > tree_filter['max_ampl']:
            tree_mask[cc_mask] = False

        # If number_of_returns is too low, it's not a tree.
        elif np.percentile(nor[cc_mask], tree_filter['nor_perc']) < tree_filter['min_nor']:
            tree_mask[cc_mask] = False

        # Look at shape of cluster, e.g. minimum bounding rectangle + min_width check.
        # min_dim, _ = helpers.get_wl_box(xyz[cc_mask])
        elif helpers.get_wl_box(xyz[cc_mask])[0] < tree_filter['min_width']:
            tree_mask[cc_mask] = False

        # If the object is smaller than MIN_HEIGHT or higher than MAX_HEIGHT, it's not a tree.
        elif np.nanmax(cc_hag) < tree_filter['minmax_hag']:
            tree_mask[cc_mask] = False

    ## Set labels
    labels = np.ones((len(xyz),), dtype='uint16') * UNKNOWN
    labels[tree_mask] = TREE


    ### Save the point cloud

    header = laspy.LasHeader(point_format=las.header.point_format,
                             version=las.header.version)
    header.offsets = las.header.offsets
    header.scales = las.header.scales

    new_las = laspy.LasData(header)

    new_las.points = las.points[orig_idx]

    new_las.add_extra_dim(laspy.ExtraBytesParams(name="label", type="uint16",
                                                 description="Label"))  
    new_las.add_extra_dim(laspy.ExtraBytesParams(name="orig_idx", type="uint32",
                                                 description="Original index"))  
    new_las.add_extra_dim(laspy.ExtraBytesParams(name="hag", type="float",
                                                 description="Height above ground"))
    # new_las.add_extra_dim(laspy.ExtraBytesParams(name="normal_x", type="float",
    #                                              description="normal_x"))
    # new_las.add_extra_dim(laspy.ExtraBytesParams(name="normal_y", type="float",
    #                                              description="normal_y"))    
    # new_las.add_extra_dim(laspy.ExtraBytesParams(name="normal_z", type="float",
    #                                              description="normal_z"))

    new_las.label = labels
    new_las.orig_idx = orig_idx
    new_las.hag = hag
    # new_las.normal_x = normals[:,0]
    # new_las.normal_y = normals[:,1]
    # new_las.normal_z = normals[:,2]

    new_las.write(output_dir / f'trees_{tilecode}.laz')